<a href="https://colab.research.google.com/github/NameMeLeo/Simple-YT-download/blob/main/Simple_YT_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# @markdown Press the play buton on left to run this cell, then click on the share link
!pip install -q gradio==4.44.1 pytubefix python-multipart==0.0.12
import gradio as gr
from pytubefix import YouTube
import pandas as pd
import os, zipfile
from datetime import datetime

def download_video(url):
    try:
        yt = YouTube(url)
        stream = yt.streams.get_highest_resolution()
        file_path = stream.download()
        return file_path  # Return the path for single download
    except Exception as e:
        return str(e)

#create a temporary directory to store the downloaded videos then zip it
def download_videos(urls):
    zip_filename = datetime.now().strftime('%H%M%S') + '.zip'
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for url in urls:
            try:
                yt = YouTube(url)
                stream = yt.streams.get_highest_resolution()
                file_path = stream.download()
                zipf.write(file_path, os.path.basename(file_path))
                os.remove(file_path)  # Remove the file after adding to zip
            except Exception as e:
                print(f"Error downloading {url}: {e}")
    return zip_filename

def process_file(file_path):
    if file_path.name.endswith('.txt'):
        with open(file_path.name, 'r') as f:
            urls = f.read().splitlines()
    elif file_path.name.endswith(('.xls', '.xlsx')):
        df = pd.read_excel(file_path.name)
        urls = df.iloc[:, 0].tolist()  # Assuming URLs are in the first column
    else:
        return "Unsupported file type."

    return download_videos(urls)

with gr.Blocks(title='YouTube Video Downloader') as demo:
    gr.Markdown("# YouTube Video Downloader")
    gr.Markdown("Paste a YouTube link or upload a .txt file containing links to download videos.")

    with gr.Tab('Simple download'):
      url_input = gr.Textbox(label="YouTube URL", placeholder="Enter a single YouTube URL")
      download_button = gr.Button("Download Video")
    with gr.Tab('Batch download'):
      upload_input = gr.File(label="Upload .txt file", file_types=['.txt'])
      upload_button = gr.Button("Download All Videos")

    output = gr.File(label="Video Download")
    download_button.click(download_video, inputs=url_input, outputs=output)
    upload_button.click(process_file, inputs=upload_input, outputs=output)

demo.launch(share=True, inline=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3c4f6c98e10cf8ad18.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
